textual inversion

In [ ]:
%cd /content/drive/MyDrive/diffusers

/content/drive/MyDrive/diffusers


In [ ]:
!pip install .

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "/content/drive/MyDrive/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipe.load_textual_inversion("/content/drive/MyDrive/shanshuifine")

prompt = "a painting of mountains and rivers in <sks-style>"


for i in range(100):

    image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
    save_path = f"/content/drive/MyDrive/shanshui/textinversion11_{i+1}.png"
    image.save(save_path)
    print(f"Image {i+1} saved to {save_path}")

dreambooth lora


In [ ]:
!pip install  -U PEFT  transformers

In [ ]:
from diffusers import StableDiffusionPipeline
import torch


pipe = StableDiffusionPipeline.from_pretrained("/content/drive/MyDrive/stable-diffusion-v1-4", torch_dtype=torch.float16).to("cuda")
pipe.unet.load_attn_procs("/content/drive/MyDrive/dreamfine/pytorch_lora_weights.safetensors")



for i in range(100):
    image = pipe("a painting of mountains and rivers in sks style", num_inference_steps=30).images[0]
    save_path = f"/content/drive/MyDrive/shanshui/dreamlora11_{i+1}.png"
    image.save(save_path)
    print(f"Image {i+1} saved to {save_path}")

dreambooth


In [ ]:
from diffusers import StableDiffusionPipeline
import torch


model_id = "/content/drive/MyDrive/dreamfine2"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "a painting of mountains and rivers in sks style"


for i in range(100):

    image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
    save_path = f"/content/drive/MyDrive/shanshui/dream11_{i+1}.png"
    image.save(save_path)
    print(f"Image {i+1} saved to {save_path}")

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

folder_path = '/content/drive/MyDrive/shanshui'

image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg', 'png', 'gif', 'bmp'))]

rows, cols = 3, 5

fig, axes = plt.subplots(rows, cols, figsize=(15, 9))

for i, ax in enumerate(axes.flat):
    if i < len(image_files):
        img_path = os.path.join(folder_path, image_files[i])
        img = Image.open(img_path)
        ax.imshow(img)
        ax.set_title(os.path.basename(image_files[i]), fontsize=8)
    ax.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

CLIP score

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


inception_model = models.inception_v3(pretrained=True, transform_input=False)
inception_model.eval()
inception_model.to(device)


transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        if img is not None:
            img = transform(img)
            images.append(img.unsqueeze(0))
    return images

def is_score(imgs, inception_model):
    with torch.no_grad():
        imgs = torch.cat(imgs, 0).to(device)
        outputs = inception_model(imgs)
    p_yx = F.softmax(outputs, dim=1)
    p_y = p_yx.mean(0).unsqueeze(0)
    kl = p_yx * (torch.log(p_yx) - torch.log(p_y))
    kl = kl.sum(1).mean()
    return torch.exp(kl).item()


folder_path = '/content/drive/MyDrive/textdata'
folder_path1 = '/content/drive/MyDrive/shanshui'
folder_path2 = '/content/drive/MyDrive/dreamwloradata'

imgs = load_images_from_folder(folder_path)
score = is_score(imgs, inception_model)
imgs1 = load_images_from_folder(folder_path1)
score1 = is_score(imgs1, inception_model)
imgs2 = load_images_from_folder(folder_path2)
score2 = is_score(imgs2, inception_model)
print(f"Textual inversion Inception Score: {score}")
print(f"Dreambooth Inception Score: {score1}")
print(f"Dreamboothlora Inception Score: {score2}")

Textual inversion Inception Score: 2.573744773864746
Dreambooth Inception Score: 2.3677496910095215
Dreamboothlora Inception Score: 2.633596897125244


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/MyDrive/Transfer/result/face2.jpg/face2单张原模型cartoon.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a man",'a man in cartoon style']).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.4648 0.535 ]]


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/MyDrive/Transfer/result/face2.jpg/face2微调模型单张cartoon.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a man",'a man in cartoon style']).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.01133 0.989  ]]


In [4]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/MyDrive/Transfer/result/building2.jpg/building2原模型单张cartoon.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a white builiding",'a white building in cartoon style']).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.5 0.5]]


In [3]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/MyDrive/Transfer/result/building2.jpg/building调模型单张cartoon.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a white builiding",'a white building in cartoon style']).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.4688 0.5312]]
